In [6]:
# Load US listed stock symbols and their listing market/exchange from NASDAQ Trader files
import pandas as pd

nasdaq_url = "https://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
other_url = "https://ftp.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"

nasdaq_raw = pd.read_csv(nasdaq_url, sep="|", dtype=str)
other_raw = pd.read_csv(other_url, sep="|", dtype=str)

nasdaq_raw.head(), other_raw.head()

URLError: <urlopen error [Errno 110] Connection timed out>

In [4]:
# Build a US stock universe (symbols + market) from Wikipedia index constituents (fallback when NASDAQ Trader FTP is unreachable)
# Then we'll enrich industry using yfinance in a later cell.
import pandas as pd
import io
import requests

sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
nasdaq100_url = "https://en.wikipedia.org/wiki/Nasdaq-100"
dow30_url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"

# Wikipedia often blocks default Python user agents; fetch HTML with a browser-like UA then parse.
WIKI_HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

def read_html_url(url: str, **read_html_kwargs):
    r = requests.get(url, headers=WIKI_HEADERS, timeout=30)
    r.raise_for_status()
    # Use bytes to avoid encoding edge cases
    return pd.read_html(io.BytesIO(r.content), **read_html_kwargs)

# S&P 500 table
sp500_tbls = read_html_url(sp500_url, match="Symbol")
sp500 = sp500_tbls[0].copy()
sp500 = sp500.rename(columns={"Symbol": "symbol", "Security": "name"})
sp500["market"] = "NYSE/NASDAQ"  # mixed; we'll refine via yfinance later if needed
sp500 = sp500[["symbol", "name", "market"]]

# NASDAQ-100: there are multiple tables; pick the one that has 'Ticker'
ndx_tbls = read_html_url(nasdaq100_url)
ndx = None
for t in ndx_tbls:
    cols = [c.lower() for c in t.columns.astype(str)]
    if "ticker" in cols and ("company" in cols or "name" in cols):
        ndx = t.copy()
        break
if ndx is None:
    raise ValueError("Could not find NASDAQ-100 constituents table with a Ticker column")

# Normalize column names
colmap = {c: c.lower() for c in ndx.columns}
ndx = ndx.rename(columns=colmap)
name_col = "company" if "company" in ndx.columns else ("name" if "name" in ndx.columns else None)
ndx = ndx.rename(columns={"ticker": "symbol", name_col: "name"})
ndx["market"] = "NASDAQ"
ndx = ndx[["symbol", "name", "market"]]

# Dow 30 constituents table
dow_tbls = read_html_url(dow30_url)
dow = None
for t in dow_tbls:
    cols = [c.lower() for c in t.columns.astype(str)]
    if any("symbol" == c for c in cols) and any("company" in c or "component" in c for c in cols):
        dow = t.copy()
        break
if dow is None:
    # fallback: just take first table and try common columns
    dow = dow_tbls[0].copy()

# Try to map likely columns
lower_cols = {c: str(c).lower() for c in dow.columns}
dow = dow.rename(columns={c: lower_cols[c] for c in dow.columns})
name_col = None
for cand in ["company", "component", "name"]:
    if cand in dow.columns:
        name_col = cand
        break
sym_col = "symbol" if "symbol" in dow.columns else ("ticker" if "ticker" in dow.columns else None)
if sym_col is None or name_col is None:
    raise ValueError(f"Could not normalize Dow table columns: {dow.columns.tolist()}")

dow = dow.rename(columns={sym_col: "symbol", name_col: "name"})
dow["market"] = "NYSE/NASDAQ"
dow = dow[["symbol", "name", "market"]]

# Combine and de-duplicate
universe = pd.concat([sp500, ndx, dow], ignore_index=True)
universe["symbol"] = universe["symbol"].astype(str).str.strip().str.replace(".", "-", regex=False)  # BRK.B -> BRK-B
universe["name"] = universe["name"].astype(str).str.strip()
universe = universe.dropna(subset=["symbol"]).drop_duplicates(subset=["symbol"], keep="first").reset_index(drop=True)

universe.head(), universe.shape

(  symbol                 name       market
 0    MMM                   3M  NYSE/NASDAQ
 1    AOS          A. O. Smith  NYSE/NASDAQ
 2    ABT  Abbott Laboratories  NYSE/NASDAQ
 3   ABBV               AbbVie  NYSE/NASDAQ
 4    ACN            Accenture  NYSE/NASDAQ,
 (517, 3))

In [5]:
# Enrich the universe with industry (and sector) using yfinance; then build an index-by-name CSV
import time
import numpy as np
import yfinance as yf

# Work on a copy to avoid mutating upstream data unexpectedly
univ = universe.copy()

# yfinance can be rate-limited; keep batches modest
symbols = univ["symbol"].astype(str).unique().tolist()

industry_map = {}
sector_map = {}

batch_size = 50
sleep_s = 1.0

for i in range(0, len(symbols), batch_size):
    batch = symbols[i:i+batch_size]
    tickers = yf.Tickers(" ".join(batch))
    # tickers.tickers is a dict symbol->Ticker
    for sym, tkr in tickers.tickers.items():
        try:
            info = tkr.fast_info  # fast_info doesn't include industry; try .info fallback
            _ = info  # no-op
            full = tkr.info
            industry_map[sym] = full.get("industry")
            sector_map[sym] = full.get("sector")
        except Exception:
            industry_map[sym] = None
            sector_map[sym] = None
    time.sleep(sleep_s)

univ["industry"] = univ["symbol"].map(industry_map)
univ["sector"] = univ["symbol"].map(sector_map)

# Create output indexed by stock name as requested.
# If names collide (e.g., share classes / similar names), keep first and warn via a quick count.
name_dupes = univ["name"].value_counts()
num_dup_names = int((name_dupes > 1).sum())

out = univ[["name", "industry", "market", "symbol", "sector"]].copy()
out = out.sort_values(["name", "symbol"], kind="stable")

# Set index to name; if duplicates exist, keep all rows (index not unique) to avoid dropping data.
out = out.set_index("name")

csv_path = "stocks_market_industry_indexed_by_name.csv"
out.to_csv(csv_path)

csv_path, out.shape, num_dup_names, out.head(10)

('stocks_market_industry_indexed_by_name.csv',
 (517, 4),
 0,
                                                 industry       market symbol  \
 name                                                                           
 3M                                         Conglomerates  NYSE/NASDAQ    MMM   
 A. O. Smith               Specialty Industrial Machinery  NYSE/NASDAQ    AOS   
 AES Corporation                  Utilities - Diversified  NYSE/NASDAQ    AES   
 APA Corporation                            Oil & Gas E&P  NYSE/NASDAQ    APA   
 ASML Holding         Semiconductor Equipment & Materials       NASDAQ   ASML   
 AT&T                                    Telecom Services  NYSE/NASDAQ      T   
 AbbVie                      Drug Manufacturers - General  NYSE/NASDAQ   ABBV   
 Abbott Laboratories                      Medical Devices  NYSE/NASDAQ    ABT   
 Accenture                Information Technology Services  NYSE/NASDAQ    ACN   
 Adobe Inc.                        Software - A

In [7]:
# Try to pull broader US listings from an alternate HTTPS source (Stooq symbol lists)
import pandas as pd
import requests
import io

stooq_sources = {
    "NYSE": "https://stooq.com/q/l/?s=NYSE&f=sd2t2ohlcvn&h&e=csv",
    "NASDAQ": "https://stooq.com/q/l/?s=NASDAQ&f=sd2t2ohlcvn&h&e=csv",
    "AMEX": "https://stooq.com/q/l/?s=AMEX&f=sd2t2ohlcvn&h&e=csv",
}

stooq_frames = {}
for mkt, url in stooq_sources.items():
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        df = pd.read_csv(io.StringIO(r.text))
        stooq_frames[mkt] = df
        print(mkt, df.shape, df.columns.tolist()[:10])
        display(df.head(3))
    except Exception as e:
        print("Failed", mkt, url, type(e).__name__, str(e)[:200])

list(stooq_frames.keys())

NYSE (1, 9) ['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name']


,Symbol,Date,Time,Open,High,Low,Close,Volume,Name
0,NYSE,N/D,N/D,N/D,N/D,N/D,N/D,N/D,NYSE


NASDAQ (1, 9) ['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name']


,Symbol,Date,Time,Open,High,Low,Close,Volume,Name
0,NASDAQ,N/D,N/D,N/D,N/D,N/D,N/D,N/D,NASDAQ


AMEX (1, 9) ['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name']


,Symbol,Date,Time,Open,High,Low,Close,Volume,Name
0,AMEX,N/D,N/D,N/D,N/D,N/D,N/D,N/D,AMEX


['NYSE', 'NASDAQ', 'AMEX']

In [8]:
# Pull a broad US ticker universe from SEC's company_tickers.json (CIK->ticker) as a starting point
# (This provides many more tickers; we'll later map them to exchanges/markets and industries.)
import pandas as pd
import requests

sec_url = "https://www.sec.gov/files/company_tickers.json"
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; stock-universe-builder/1.0; +https://example.com)",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "application/json",
}

r = requests.get(sec_url, headers=headers, timeout=30)
r.raise_for_status()
raw = r.json()

sec = pd.DataFrame.from_dict(raw, orient="index")
sec = sec.rename(columns={"cik_str": "cik", "ticker": "symbol", "title": "name"})
sec["symbol"] = sec["symbol"].astype(str).str.strip().str.replace(".", "-", regex=False)
sec["name"] = sec["name"].astype(str).str.strip()

sec.shape, sec.head(5)

((10382, 3),
        cik symbol            name
 0  1045810   NVDA     NVIDIA CORP
 1   320193   AAPL      Apple Inc.
 2  1652044  GOOGL   Alphabet Inc.
 3   789019   MSFT  MICROSOFT CORP
 4  1018724   AMZN  AMAZON COM INC)

In [10]:
# Enrich a broader set of tickers from SEC with exchange/market + industry/sector via yfinance
# We'll sample to keep runtime reasonable and avoid rate limits, and only keep tickers that yfinance recognizes.
import pandas as pd
import numpy as np
import time
import yfinance as yf

# Start from SEC tickers; drop obvious non-common/odd tickers if desired later
sec_base = sec.copy()

# Prefer tickers not already in our existing index-based universe
existing_syms = set(univ["symbol"].astype(str).unique())
sec_new = sec_base[~sec_base["symbol"].isin(existing_syms)].copy()

# Sample to add 'more unknown' names without pulling all 10k at once
# You can increase this number if you want a larger file.
N_ADD = 1500
sec_new = sec_new.sample(n=min(N_ADD, len(sec_new)), random_state=42).reset_index(drop=True)

# Batched yfinance pulls
batch_size = 100
sleep_s = 1.0

rows = []
for i in range(0, len(sec_new), batch_size):
    batch = sec_new.loc[i:i+batch_size-1, "symbol"].astype(str).tolist()
    tickers = yf.Tickers(" ".join(batch))
    for sym, tkr in tickers.tickers.items():
        # yfinance may raise noisy HTTPError (404 "quote not found") while populating `.info`.
        # We treat those as "symbol not supported" and skip.
        try:
            info = tkr.info or {}
        except Exception:
            continue

        try:
            name = (
                info.get("shortName")
                or info.get("longName")
                or sec_new.loc[sec_new["symbol"] == sym, "name"].head(1).squeeze()
            )
            industry = info.get("industry")
            sector = info.get("sector")
            # Exchange/market: yfinance uses keys like exchange, fullExchangeName, exchangeTimezoneName
            exch = info.get("exchange") or info.get("fullExchangeName")

            rows.append(
                {
                    "symbol": sym,
                    "name": name,
                    "market": exch,
                    "industry": industry,
                    "sector": sector,
                    "source": "SEC+yfinance",
                }
            )
        except Exception:
            continue
    time.sleep(sleep_s)

sec_enriched = pd.DataFrame(rows)

# Combine with our existing 'univ' (index-derived) universe
idx_univ = univ.copy()
idx_univ["source"] = "Indices+yfinance"

combined = pd.concat([idx_univ[["symbol","name","market","industry","sector","source"]], sec_enriched], ignore_index=True)

# Clean, drop obvious empties, de-dupe by symbol preferring the index-derived rows first
combined["symbol"] = combined["symbol"].astype(str).str.strip()
combined["name"] = combined["name"].astype(str).str.strip()
combined = combined.replace({"name": {"nan": np.nan}, "industry": {"nan": np.nan}, "sector": {"nan": np.nan}, "market": {"nan": np.nan}})
combined = combined.dropna(subset=["symbol", "name"])

# Prefer existing index-universe row if duplicate symbol
combined["source_rank"] = combined["source"].map({"Indices+yfinance": 0, "SEC+yfinance": 1}).fillna(9)
combined = combined.sort_values(["symbol", "source_rank"], kind="stable")
combined = combined.drop_duplicates(subset=["symbol"], keep="first").drop(columns=["source_rank"])

# Create output indexed by company name; industry should be 2nd column
expanded_out = combined[["name", "industry", "market", "symbol", "sector", "source"]].sort_values(["name","symbol"], kind="stable").set_index("name")

expanded_csv_path = "stocks_market_industry_indexed_by_name_EXPANDED.csv"
expanded_out.to_csv(expanded_csv_path)

expanded_csv_path, expanded_out.shape, expanded_out.head(10)

('stocks_market_industry_indexed_by_name_EXPANDED.csv',
 (517, 5),
                                                 industry       market symbol  \
 name                                                                           
 3M                                         Conglomerates  NYSE/NASDAQ    MMM   
 A. O. Smith               Specialty Industrial Machinery  NYSE/NASDAQ    AOS   
 AES Corporation                  Utilities - Diversified  NYSE/NASDAQ    AES   
 APA Corporation                            Oil & Gas E&P  NYSE/NASDAQ    APA   
 ASML Holding         Semiconductor Equipment & Materials       NASDAQ   ASML   
 AT&T                                    Telecom Services  NYSE/NASDAQ      T   
 AbbVie                      Drug Manufacturers - General  NYSE/NASDAQ   ABBV   
 Abbott Laboratories                      Medical Devices  NYSE/NASDAQ    ABT   
 Accenture                Information Technology Services  NYSE/NASDAQ    ACN   
 Adobe Inc.                        Softwar

In [11]:
# Add a required set of common ETFs/funds + user-provided tickers, enrich via yfinance, and export an updated CSV
import pandas as pd
import numpy as np
import yfinance as yf

# Tickers explicitly mentioned by user (including some non-stocks like ETFs and cash placeholders)
user_tickers = [
    "BZFD", "STKL", "OGN", "SG", "INTC", "PTON", "NXT", "IHI", "RIVN", "RKLB", "IAU",
    "NFLX", "MRNA", "AMZN", "IBM", "O", "GLD", "MSFT", "VUG", "SONO", "AMD", "VOOV",
]

# Common ETFs/funds a typical investor may hold (US)
common_funds = [
    # Broad market
    "SPY", "VOO", "IVV", "VTI", "ITOT", "SCHB",
    # S&P 500 / total market mutual funds (some may not resolve on yfinance; harmless)
    "VFIAX", "FXAIX", "SWPPX", "VTSAX", "FSKAX", "SWTSX",
    # Growth / value
    "QQQ", "VGT", "SCHG", "VUG", "VOOG", "VOOV", "VTV", "SCHV",
    # Dividends
    "VYM", "SCHD", "DGRO",
    # Bonds
    "BND", "AGG", "LQD", "TLT", "IEF", "SHY",
    # International
    "VXUS", "VEA", "VWO", "IEFA", "IEMG",
    # Gold/commodities
    "GLD", "IAU", "SLV",
    # Real estate
    "VNQ", "SCHH",
    # Sector examples
    "XLK", "XLF", "XLV", "XLE", "XLY", "XLP",
]

required_tickers = sorted(set([t.upper() for t in (user_tickers + common_funds) if t and t.upper() not in {"CASH", "ACCOUNT:"}]))

# Fetch metadata from yfinance (batched)
batch_size = 50
rows = []
for i in range(0, len(required_tickers), batch_size):
    batch = required_tickers[i:i+batch_size]
    tickers = yf.Tickers(" ".join(batch))
    for sym, tkr in tickers.tickers.items():
        try:
            info = tkr.info or {}
        except Exception:
            continue

        name = info.get("shortName") or info.get("longName") or info.get("displayName")
        industry = info.get("industry")
        sector = info.get("sector")
        market = info.get("exchange") or info.get("fullExchangeName")

        # Some funds don't have industry/sector; keep them as NaN rather than string "None"
        rows.append(
            {
                "symbol": sym,
                "name": name,
                "market": market,
                "industry": industry,
                "sector": sector,
                "source": "RequiredTickers+yfinance",
            }
        )

req_df = pd.DataFrame(rows)

# Combine with existing expanded_out/combined (already deduped by symbol)
base = combined.copy() if "combined" in globals() else idx_univ.copy()

# Normalize
req_df["symbol"] = req_df["symbol"].astype(str).str.strip()
req_df["name"] = req_df["name"].astype(str).str.strip()
req_df = req_df.replace({"name": {"nan": np.nan}, "industry": {"nan": np.nan}, "sector": {"nan": np.nan}, "market": {"nan": np.nan}})
req_df = req_df.dropna(subset=["symbol", "name"]).drop_duplicates(subset=["symbol"], keep="first")

base["symbol"] = base["symbol"].astype(str).str.strip()
base["name"] = base["name"].astype(str).str.strip()

combo2 = pd.concat([base, req_df], ignore_index=True)
# Prefer base rows first, then required tickers for anything missing
combo2["source_rank"] = combo2["source"].map({"Indices+yfinance": 0, "SEC+yfinance": 1, "RequiredTickers+yfinance": 2}).fillna(9)
combo2 = combo2.sort_values(["symbol", "source_rank"], kind="stable").drop_duplicates(subset=["symbol"], keep="first")
combo2 = combo2.drop(columns=["source_rank"])

# Output indexed by name with industry as 2nd column
final_out = combo2[["name", "industry", "market", "symbol", "sector", "source"]].sort_values(["name", "symbol"], kind="stable").set_index("name")

final_csv_path = "stocks_market_industry_indexed_by_name_WITH_FUNDS.csv"
final_out.to_csv(final_csv_path)

final_csv_path, final_out.shape, req_df.shape, final_out.head(10)

('stocks_market_industry_indexed_by_name_WITH_FUNDS.csv',
 (572, 5),
 (63, 6),
                                                 industry       market symbol  \
 name                                                                           
 3M                                         Conglomerates  NYSE/NASDAQ    MMM   
 A. O. Smith               Specialty Industrial Machinery  NYSE/NASDAQ    AOS   
 AES Corporation                  Utilities - Diversified  NYSE/NASDAQ    AES   
 APA Corporation                            Oil & Gas E&P  NYSE/NASDAQ    APA   
 ASML Holding         Semiconductor Equipment & Materials       NASDAQ   ASML   
 AT&T                                    Telecom Services  NYSE/NASDAQ      T   
 AbbVie                      Drug Manufacturers - General  NYSE/NASDAQ   ABBV   
 Abbott Laboratories                      Medical Devices  NYSE/NASDAQ    ABT   
 Accenture                Information Technology Services  NYSE/NASDAQ    ACN   
 Adobe Inc.                   

In [12]:
# Define an additional curated list of ~100 widely-held ETFs/stocks to include
import pandas as pd

additional_required = [
    # Mega-cap / common stocks
    "GOOG", "META", "BRK-B", "TSLA", "JPM", "JNJ", "V", "MA", "PG", "UNH",
    "HD", "COST", "KO", "PEP", "AVGO", "NVDA", "LLY", "XOM", "CVX", "WMT",
    "DIS", "ADBE", "CRM", "ORCL", "CSCO", "QCOM", "TXN", "INTU", "AMAT", "MU",
    "CAT", "BA", "GE", "MMM", "MCD", "NKE", "SBUX", "TMO", "ABT", "PFE",
    "MRK", "ABBV", "MDT", "DHR", "RTX", "LMT", "UPS", "FDX", "BKNG", "UBER",

    # Broad US equity ETFs
    "IWM", "DIA", "MDY", "VV", "VXF", "RSP", "SPLG", "IVOO", "IJH", "IJR",
    "VO", "VB", "SCHX", "SCHM", "SCHA", "IVOG", "IVOV", "IJK", "IJS", "IWB",

    # Factor / style ETFs
    "VBR", "VOT", "VONV", "VONE", "IWF", "IWD", "MTUM", "QUAL", "USMV", "SPLV",
    "VLUE", "IUSG", "IUSV", "SCHG", "SCHV", "DGRO", "HDV", "SPHD", "NOBL", "SDY",

    # Tech / thematic ETFs
    "VCR", "VDC", "VFH", "VDE", "VHT", "VIS", "VAW", "VPU", "VGT", "SOXX",
    "SMH", "IGV", "HACK", "CIBR", "ARKK", "ARKG", "BOTZ", "ROBO", "ICLN", "TAN",

    # International equity ETFs
    "EFA", "EEM", "EWJ", "EWU", "EWG", "EWQ", "EWC", "MCHI", "INDA", "EWZ",
    "FXI", "KWEB", "VPL", "IPAC", "SCZ", "SCHF", "SCHF", "SCHC", "EMB", "VYMI",

    # Fixed income / cash-like / credit ETFs
    "TIP", "VTIP", "BIL", "SHV", "SGOV", "MINT", "JPST", "NEAR", "HYG", "JNK",
    "VCIT", "VCSH", "IGIB", "IEF", "IEI", "TLH", "EDV", "MUB", "TFI", "SHM",

    # Real assets / REITs / commodities
    "VNQI", "IYR", "SCHH", "REET", "DBC", "PDBC", "USO", "UNG", "CPER", "DBA",
]

# De-duplicate and normalize (yfinance uses '-' for share classes like BRK-B)
additional_required = sorted(set([t.strip().upper() for t in additional_required if isinstance(t, str) and t.strip()]))

len(additional_required), additional_required[:20]

(159,
 ['ABBV',
  'ABT',
  'ADBE',
  'AMAT',
  'ARKG',
  'ARKK',
  'AVGO',
  'BA',
  'BIL',
  'BKNG',
  'BOTZ',
  'BRK-B',
  'CAT',
  'CIBR',
  'COST',
  'CPER',
  'CRM',
  'CSCO',
  'CVX',
  'DBA'])

In [13]:
# Enrich the additional curated ticker list via yfinance and export an updated CSV
import pandas as pd
import numpy as np
import time
import yfinance as yf

# Use additional_required defined in prior cell
add_tickers = additional_required

batch_size = 50
sleep_s = 0.8

add_rows = []
failed = []

for i in range(0, len(add_tickers), batch_size):
    batch = add_tickers[i:i+batch_size]
    tickers = yf.Tickers(" ".join(batch))
    for sym, tkr in tickers.tickers.items():
        try:
            info = tkr.info or {}
        except Exception:
            failed.append(sym)
            continue

        name = info.get("shortName") or info.get("longName") or info.get("displayName")
        market = info.get("exchange") or info.get("fullExchangeName")
        industry = info.get("industry")
        sector = info.get("sector")

        if not name:
            failed.append(sym)
            continue

        add_rows.append(
            {
                "symbol": sym,
                "name": str(name).strip(),
                "market": market,
                "industry": industry,
                "sector": sector,
                "source": "AdditionalRequired+yfinance",
            }
        )

    time.sleep(sleep_s)

add_df = pd.DataFrame(add_rows)

# Combine with existing combo2 (already includes indices + some required tickers)
base2 = combo2.copy()

# Normalize + de-dupe additional rows
if not add_df.empty:
    add_df["symbol"] = add_df["symbol"].astype(str).str.strip()
    add_df["name"] = add_df["name"].astype(str).str.strip()
    add_df = add_df.replace({"name": {"nan": np.nan}, "industry": {"nan": np.nan}, "sector": {"nan": np.nan}, "market": {"nan": np.nan}})
    add_df = add_df.dropna(subset=["symbol", "name"]).drop_duplicates(subset=["symbol"], keep="first")

combo3 = pd.concat([base2, add_df], ignore_index=True)
combo3["source_rank"] = combo3["source"].map(
    {"Indices+yfinance": 0, "SEC+yfinance": 1, "RequiredTickers+yfinance": 2, "AdditionalRequired+yfinance": 3}
).fillna(9)
combo3 = combo3.sort_values(["symbol", "source_rank"], kind="stable").drop_duplicates(subset=["symbol"], keep="first")
combo3 = combo3.drop(columns=["source_rank"])

# Output indexed by name with industry as 2nd column
final_out2 = combo3[["name", "industry", "market", "symbol", "sector", "source"]].sort_values(
    ["name", "symbol"], kind="stable"
).set_index("name")

final_csv_path2 = "stocks_market_industry_indexed_by_name_WITH_FUNDS_PLUS_100.csv"
final_out2.to_csv(final_csv_path2)

# Summarize resolution
resolved = int(add_df["symbol"].nunique()) if not add_df.empty else 0
requested = len(add_tickers)

final_csv_path2, final_out2.shape, requested, resolved, (requested - resolved), failed[:25]

('stocks_market_industry_indexed_by_name_WITH_FUNDS_PLUS_100.csv',
 (675, 5),
 159,
 159,
 0,
 [])